<a href="https://colab.research.google.com/github/misbahsy/APMonitor-do/blob/master/MultiObjectiveOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multi-Objective Optimization

<p class='vspace'>Many optimization problems have multiple competing objectives. These competing objectives are part of the trade-off that defines an optimal solution. Sometimes these competing objectives have separate priorities where one objective should be satisfied <em>before</em> another objective is even considered. This especially arises in model predictive control or other types of dynamic optimization problems. There are competing objectives with a ranked hierarchy. The highest level objectives are satisfied first followed by lower ranked objectives if there are additional degrees of freedom available. The l1-norm objective is a natural way to explicitly rank objectives and simultaneously optimize multiple priorities with a single optimization problem.
</p>


##Exercise
<p>Consider examples of safety, environmental, and economic constraints or objectives. Which are most important and why? 
</p>
<p class='vspace'>For the following multi-objective optimization problem, sketch a possible optimal trajectory.
</p>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/multi-objective_control.pdf' rel='nofollow'>Multiple Objectives Exercise (pdf)</a></div>
<div class='vspace'></div><div><img src='http://apmonitor.com/do/uploads/Main/multi-objective_control.png' alt='' title='' /></div>


##Solution
<div><img src='http://apmonitor.com/do/uploads/Main/download.png' alt='' title='' /> <a class='urllink' href='http://apmonitor.com/do/uploads/Main/multi-objective_control.zip' rel='nofollow'>Multi-Objective Model Predictive Control</a></div>
<div class='vspace'></div>

In [1]:
#@title
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/JlMVZmGgjDY" frameborder="0" allowfullscreen></iframe>